In [1]:
# Bloque 1: Importaciones y Configuración
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# --- CONFIGURACIÓN DE PATHS (Ajustado a tu estructura) ---
DATASET_BASE_PATH = "audios_proyecto/"
#DATASET_BASE_PATH ="Users\DIEGO\Documents\OptativoAcus2025S2\acus220_primavera_2025/audios/audios_proyecto/"
CLASES = ["pequeno", "mediano", "grande"]

In [2]:
# Bloque 2: Función de Extracción de Características
def extract_features(file_path, n_mfcc=40):
    """Carga un archivo de audio (path) y extrae la media de 40 MFCCs."""
    try:
        y, sr = librosa.load(file_path, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        features_vector = np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Error cargando {file_path}: {e}")
        return None
    return features_vector

In [3]:
# Bloque 3: Carga y Extracción de Datos
features = [] 
labels = []   

print("1. Cargando y extrayendo features...")
for i, clase in enumerate(CLASES):
    folder_path = os.path.join(DATASET_BASE_PATH, clase)
    print(f"   -> Procesando clase: {clase} (etiqueta: {i})")
    
    if not os.path.isdir(folder_path):
        print(f"   ERROR: Carpeta no encontrada: {folder_path}.")
        continue
        
    for filename in os.listdir(folder_path):
        if filename.endswith(('.wav', '.mp3')):
            file_path = os.path.join(folder_path, filename)
            data = extract_features(file_path) # Usa la función del Bloque 2
            
            if data is not None:
                features.append(data)
                labels.append(i)

if not features:
    print("\n¡ADVERTENCIA! No se cargaron muestras. Verifica tus carpetas.")
else:
    print(f"\nTotal de muestras cargadas: {len(features)}")
    X = np.array(features)
    y = np.array(labels)

1. Cargando y extrayendo features...
   -> Procesando clase: pequeno (etiqueta: 0)
   -> Procesando clase: mediano (etiqueta: 1)
   -> Procesando clase: grande (etiqueta: 2)

Total de muestras cargadas: 28


In [4]:
# Bloque 4: Entrenamiento y Guardado del Modelo
if 'X' in locals() and len(X) > 0:
    # Dividir datos
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar
    print("2. Entrenando modelo RandomForest...")
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Evaluar
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"   Precisión en datos de prueba: {accuracy * 100:.2f}%")

    # Guardar
    joblib.dump(model, 'clasificador_vehiculos.joblib')
    print("\n3. Modelo guardado exitosamente como 'clasificador_vehiculos.joblib'")
else:
    print("\nNo se pudo entrenar el modelo (no se cargaron datos).")

2. Entrenando modelo RandomForest...
   Precisión en datos de prueba: 66.67%

3. Modelo guardado exitosamente como 'clasificador_vehiculos.joblib'
